---
layout: post
title: Sprint 2 - Random Numbers (Javascript)
description:  Random Numbers in Javascript
toc: true
breadcrumbs: True
permalink: /Random_numbersJS
---

## Fun with Random Numbers in JavaScript

Welcome! In this short interactive lesson we'll explore how to generate random numbers in JavaScript, why they're useful, and some simple techniques to make predictable (seeded) randomness for learning and games.

We'll use plain JavaScript (no external libraries) and small code snippets you can run in the browser console or a Node REPL.


### Learning objectives

- Understand how `Math.random()` works and its range.
- Learn small helper functions: random integer, random float in range, and choice from an array.
- See a reproducible (seeded) pseudo-random generator for testing and games.
- Try a short simulation: dice rolls and a randomized name picker.

In [ ]:
// Basic Math.random() examples
// 1) A random float in [0, 1)
console.log('random float [0,1):', Math.random());

// 2) Random float in [min, max)
function randomFloat(min, max) {
  return Math.random() * (max - min) + min;
}
console.log('random float [5,10):', randomFloat(5, 10));

// 3) Random integer in [min, max] inclusive
function randomInt(min, max) {
  return Math.floor(Math.random() * (max - min + 1)) + min;
}
console.log('random int 1-6 (dice):', randomInt(1, 6));

// 4) Random element from an array
function choice(arr) {
  if (!arr.length) return undefined;
  return arr[Math.floor(Math.random() * arr.length)];
}
console.log('choice from names:', choice(['Ada', 'Grace', 'Linus', 'Alan']));

### Seeded pseudo-random (LCG) — reproducible runs

JavaScript's `Math.random()` can't be seeded directly, but we can create a simple linear congruential generator (LCG) for classroom use. It's predictable — which is great for tests and demos.

In [ ]:
// Simple LCG implementation (not cryptographically secure)
function makeLCG(seed) {
  let state = seed >>> 0; // ensure 32-bit unsigned
  const a = 1664525;
  const c = 1013904223;
  const m = 2 ** 32;
  return function() {
    state = (a * state + c) >>> 0;
    return state / m; // float in [0,1)
  };
}

const rngA = makeLCG(12345);
const rngB = makeLCG(12345); // same seed -> same sequence
console.log('lcg first:', rngA());
console.log('lcg second (same seed):', rngB());

// helper using LCG to get an int in [min,max]
function seededInt(rng, min, max) {
  return Math.floor(rng() * (max - min + 1)) + min;
}
console.log('seeded dice (1-6):', seededInt(rngA, 1, 6));

In [ ]:
// Simulation: roll a dice N times and show frequency distribution
function rollDiceTrials(trials, rngFn) {
  const counts = [0,0,0,0,0,0];
  for (let i = 0; i < trials; i++) {
    const roll = Math.floor(rngFn() * 6) + 1;
    counts[roll - 1]++;
  }
  return counts.map((c, i) => ({face: i+1, count: c, pct: (c / trials * 100).toFixed(2) + '%'}));
}

console.log('Random dice (Math.random), 6000 trials:', rollDiceTrials(6000, Math.random));

// Using seeded RNG to reproduce the same histogram
const seeded = makeLCG(42);
console.log('Seeded dice (seed=42), 6000 trials:', rollDiceTrials(6000, seeded));

// Quick visual hint: show counts in simple text bar
function printHistogram(counts) {
  counts.forEach(row => {
    console.log(row.face + ': ' + '#'.repeat(Math.max(1, Math.round(row.count / 100)) ) + ' ' + row.count + ' (' + row.pct + ')');
  });
}
const counts = rollDiceTrials(6000, Math.random);
printHistogram(counts);

### Try this (challenges)

1. Modify the dice simulation to simulate two dice and compute the distribution of sums (2–12).

2. Build a `shuffle(array, rng)` function (Fisher–Yates) and test it with `makeLCG` seeds to show reproducible shuffles.

3. Create a small prize draw: given a list of names, pick 3 unique winners using the seeded RNG so results are reproducible for testing.

## Homeworks & Hacks

These exercises fun — try one or more as homework or a weekend hackathon. You can copy the code into an HTML file or run snippets in the browser console.

1. Build a web dice roller with animated visuals: roll 2 dice and display images for each face; show running statistics for the last 100 rolls.

2. Create a reproducible playlist shuffler: given an array of song titles, shuffle using `makeLCG(seed)` and allow the user to pick a seed so they can share the exact playlist order.

3. Make a 'Loot Box' simulator: items have rarities, and the drop rates are defined; compute the expected number of boxes to get a legendary item and simulate the process many times to estimate the distribution.

4. Implement and test a `weightedChoice(items, weights, rng)` function to select items based on weights (e.g., 70% common, 25% rare, 5% epic).

5. Turn the prize draw into a small web app that animates names scrolling and stops at winners (use seeded RNG for reproducible demos).

## Interactive demos (copy-paste into an HTML file or the browser console)

Below are three small self-contained demos: a Dice Roller, a Name Picker, and a Shuffle Visualizer. Save each snippet as `demo.html` and open it in your browser to play and edit.

In [ ]:
<!-- Demo 1: Dice Roller (HTML + JS) -->
<!doctype html>
<html><head><meta charset="utf-8"><title>Dice Roller Demo</title></head><body>
<h2>Dice Roller</h2>
<button id="roll">Roll 2 Dice</button>
<div id="out"></div>
<script>
function randomInt(min,max){return Math.floor(Math.random()*(max-min+1))+min;}
function roll(){
  const d1=randomInt(1,6); const d2=randomInt(1,6);
  const sum=d1+d2;
  document.getElementById('out').innerHTML = `Dice: ${d1} + ${d2} = <strong>${sum}</strong>`;
}
document.getElementById('roll').addEventListener('click', roll);
</script></body></html>

In [ ]:
<!-- Demo 2: Name Picker (seeded) -->
<!doctype html>
<html><head><meta charset="utf-8"><title>Name Picker</title></head><body>
<h2>Seeded Name Picker</h2>
Seed: <input id="seed" value="42"> <button id="pick">Pick 1 Winner</button>
<div id="winner"></div>
<script>
function makeLCG(seed){let s=seed>>>0;const a=1664525,c=1013904223,m=2**32;return()=>{s=(a*s+c)>>>0;return s/m;};}
function choiceSeeded(arr, rng){return arr[Math.floor(rng()*arr.length)];}
const list = ['Ada','Grace','Linus','Alan','Margaret','Tim'];
document.getElementById('pick').onclick = ()=>{
  const seed = Number(document.getElementById('seed').value)||0;
  const rng = makeLCG(seed);
  const win = choiceSeeded(list, rng);
  document.getElementById('winner').innerText = 'Winner: ' + win;
};
</script></body></html>

In [ ]:
<!-- Demo 3: Shuffle Visualizer (Fisher–Yates) -->
<!doctype html>
<html><head><meta charset="utf-8"><title>Shuffle Visualizer</title></head><body>
<h2>Shuffle Visualizer</h2>
Seed: <input id="s" value="1"> <button id="sh">Shuffle</button>
<pre id="arr"></pre>
<script>
function makeLCG(seed){let s=seed>>>0;const a=1664525,c=1013904223;return()=>{s=(a*s+c)>>>0;return s/(2**32);};}
function shuffle(array, rng){
  const a = array.slice();
  for(let i=a.length-1;i>0;i--){
    const j=Math.floor(rng()*(i+1));
    [a[i],a[j]]=[a[j],a[i]];
  }
  return a;
}
const sample = ['A','B','C','D','E','F','G'];
document.getElementById('sh').onclick = ()=>{
  const seed=Number(document.getElementById('s').value)||0; const rng=makeLCG(seed);
  document.getElementById('arr').textContent = shuffle(sample, rng).join(', ');
};
</script></body></html>

### Optional Hacks (ONLY 1 is REQUIRED TO BE COMPLETED!)
Hack 1: Random Color Generator

Generate random RGB colors with one click
Format: rgb(random, random, random) where each is 0-255
Change the background color of a box to show the color
Display the color code so users can copy it

Hack 2: Random Password Maker

Create passwords with random letters and numbers
Let users choose password length (6-20 characters)
Use arrays: ['a','b','c'...'z','0','1'...'9']
One button generates a new password each time

Hack 3: Coin Flip Streak Tracker

Flip a coin (Heads or Tails) with random 50/50 chance
Track the longest streak of same results in a row
Example: "Heads, Heads, Heads = 3 streak!"
Show current flip and best streak ever

Hack 4: Random Compliment Generator

Pick random compliments from a list of 10-15 nice phrases
"You're awesome!", "Great job today!", "You're very smart!"
Click button to get a new random compliment
Simple mood booster app

Hack 5: Number Guessing Game

Computer picks random number 1-50
User tries to guess it
Show "Too high!" or "Too low!" hints
Count how many guesses it takes to win

Hack 6: Random Team Maker

Enter 6-10 names in a text box
Randomly split them into 2 equal teams
Use array shuffling with random numbers
Display Team A and Team B lists

All use basic Math.random(), Math.floor(), arrays, and simple HTML buttons/displays!